In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime
from sklearn.preprocessing import LabelEncoder
from math import sqrt

pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 500)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_curve, auc, roc_auc_score, roc_curve, recall_score


from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor


In [2]:
churn_df=pd.read_csv("D:/Inceptez Technologies/HACKATHON_14_JULY_2019/churn.csv")

In [3]:
churn_df.shape

(3333, 21)

In [4]:
churn_df.isna().sum()

State             0
Account Length    0
Area Code         0
Phone             0
Int'l Plan        0
VMail Plan        0
VMail Message     0
Day Mins          0
Day Calls         0
Day Charge        0
Eve Mins          0
Eve Calls         0
Eve Charge        0
Night Mins        0
Night Calls       0
Night Charge      0
Intl Mins         0
Intl Calls        0
Intl Charge       0
CustServ Calls    0
Churn?            0
dtype: int64

In [5]:
churn_df.dtypes.value_counts()

float64    8
int64      8
object     5
dtype: int64

In [6]:
churn_df=churn_df.rename(columns={'Churn?':'churn'})
churn_df=churn_df.rename(columns={"Int'l Plan":"Intl Plan"})
churn_df.columns

Index(['State', 'Account Length', 'Area Code', 'Phone', 'Intl Plan',
       'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge',
       'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls',
       'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
       'CustServ Calls', 'churn'],
      dtype='object')

In [7]:
churn_df['churn'].value_counts()

False.    2850
True.      483
Name: churn, dtype: int64

In [8]:
churn_df['churn']=churn_df['churn'].apply(lambda x:1 if x=="True." else 0 )
churn_df['Intl Plan']=churn_df['Intl Plan'].apply(lambda x:1 if x=="yes" else 0 )
churn_df['VMail Plan']=churn_df['VMail Plan'].apply(lambda x:1 if x=="yes" else 0 )
churn_df['VMail Plan'].value_counts()


0    2411
1     922
Name: VMail Plan, dtype: int64

In [9]:
churn_df.head(10)

,State,Account Length,Area Code,Phone,Intl Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,churn
0,KS,128,415,382-4657,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,OH,107,415,371-7191,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,NJ,137,415,358-1921,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,OH,84,408,375-9999,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,OK,75,415,330-6626,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0
5,AL,118,510,391-8027,1,0,0,223.4,98,37.98,220.6,101,18.75,203.9,118,9.18,6.3,6,1.70,0,0
6,MA,121,510,355-9993,0,1,24,218.2,88,37.09,348.5,108,29.62,212.6,118,9.57,7.5,7,2.03,3,0
7,MO,147,415,329-9001,1,0,0,157.0,79,26.69,103.1,94,8.76,211.8,96,9.53,7.1,6,1.92,0,0
8,LA,117,408,335-4719,0,0,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,0
9,WV,141,415,330-8173,1,1,37,258.6,84,43.96,222.0,111,18.87,326.4,97,14.69,11.2,5,3.02,0,0


In [10]:
churn_df_dropped=churn_df.drop(['State','Area Code','Phone'],axis=1)

In [11]:
churn_df_dropped.columns

Index(['Account Length', 'Intl Plan', 'VMail Plan', 'VMail Message',
       'Day Mins', 'Day Calls', 'Day Charge', 'Eve Mins', 'Eve Calls',
       'Eve Charge', 'Night Mins', 'Night Calls', 'Night Charge', 'Intl Mins',
       'Intl Calls', 'Intl Charge', 'CustServ Calls', 'churn'],
      dtype='object')

In [12]:
X=churn_df_dropped.drop(columns=['churn'])
y=churn_df_dropped[['churn']]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size = 0.2, random_state = 100)

#y_train = y_train.ravel()
#y_test = y_test.ravel()

In [14]:
# using logistic regression
from sklearn import metrics
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
train_Pred = logreg.predict(X_train)
test_pred = logreg.predict(X_test)
print("Accuracy of Logistic regression for train",metrics.accuracy_score(y_train,train_Pred))
print("Accuracy of Logistic regression for train",metrics.accuracy_score(y_test,test_pred))

C:\Users\Venkat\Anaconda3_1\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Venkat\Anaconda3_1\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy of Logistic regression for train 0.8522130532633159
Accuracy of Logistic regression for train 0.8860569715142429


In [15]:
y_train=np.ravel(y_train)
y_test=np.ravel(y_test)
accuracy_train_dict={}
accuracy_test_dict={}
df_len=round(sqrt(len(churn_df_dropped)))
for k in range(3,df_len):
    K_value = k+1
    neigh = KNeighborsClassifier(n_neighbors = K_value, weights='uniform', algorithm='auto')
    neigh.fit(X_train, y_train) 
    y_pred_train = neigh.predict(X_train)
    y_pred_test = neigh.predict(X_test)    
    train_accuracy=accuracy_score(y_train,y_pred_train)*100
    test_accuracy=accuracy_score(y_test,y_pred_test)*100
    accuracy_train_dict.update(({k:train_accuracy}))
    accuracy_test_dict.update(({k:test_accuracy}))
    print ("Accuracy for train :",train_accuracy ," and test :",test_accuracy,"% for K-Value:",K_value)

Accuracy for train : 88.9722430607652  and test : 91.30434782608695 % for K-Value: 4
Accuracy for train : 89.12228057014254  and test : 90.4047976011994 % for K-Value: 5
Accuracy for train : 88.6721680420105  and test : 91.00449775112443 % for K-Value: 6
Accuracy for train : 88.55963990997749  and test : 90.4047976011994 % for K-Value: 7
Accuracy for train : 88.44711177794449  and test : 90.4047976011994 % for K-Value: 8
Accuracy for train : 88.7846961740435  and test : 90.25487256371814 % for K-Value: 9
Accuracy for train : 87.95948987246813  and test : 90.85457271364318 % for K-Value: 10
Accuracy for train : 88.29707426856714  and test : 90.85457271364318 % for K-Value: 11
Accuracy for train : 87.69692423105776  and test : 90.70464767616193 % for K-Value: 12
Accuracy for train : 87.95948987246813  and test : 90.85457271364318 % for K-Value: 13
Accuracy for train : 87.80945236309077  and test : 90.85457271364318 % for K-Value: 14
Accuracy for train : 87.88447111777946  and test : 90.8

In [16]:
# using Naive bayes
from sklearn.naive_bayes import GaussianNB
NB=GaussianNB()
NB.fit(X_train, y_train)
train_pred=NB.predict(X_train)
test_pred=NB.predict(X_test)
print("Accuracy of Naive bayes train set",accuracy_score(train_pred,y_train))
print("Accuracy of Naive bayes test set",accuracy_score(test_pred,y_test))

Accuracy of Naive bayes train set 0.8672168042010503
Accuracy of Naive bayes test set 0.8680659670164917


In [17]:
## Boosting
import matplotlib.pyplot as plt
# Adaboost Classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
dtree = DecisionTreeClassifier(criterion='gini',max_depth=1)

adabst_fit = AdaBoostClassifier(base_estimator= dtree,
        n_estimators=5000,learning_rate=0.05,random_state=42)

adabst_fit.fit(X_train, y_train)

#print ("\nAdaBoost - Train Confusion Matrix\n\n",pd.crosstab(y_train,adabst_fit.predict(X_train),rownames = ["Actuall"],colnames = ["Predicted"]))      
print ("\nAdaBoost  - Train accuracy",round(accuracy_score(y_train,adabst_fit.predict(X_train)),3))
#print ("\nAdaBoost  - Train Classification Report\n",classification_report(y_train,adabst_fit.predict(X_train)))

#print ("\n\nAdaBoost  - Test Confusion Matrix\n\n",pd.crosstab(y_test,adabst_fit.predict(X_test),rownames = ["Actuall"],colnames = ["Predicted"]))      
print ("\nAdaBoost  - Test accuracy",round(accuracy_score(y_test,adabst_fit.predict(X_test)),3))
#print ("\nAdaBoost - Test Classification Report\n",classification_report(y_test,adabst_fit.predict(X_test)))


AdaBoost  - Train accuracy 0.9

AdaBoost  - Test accuracy 0.895


In [18]:
# Gradientboost Classifier
from sklearn.ensemble import GradientBoostingClassifier

gbc_fit = GradientBoostingClassifier(loss='deviance',learning_rate=0.05,n_estimators=5000,
                                     min_samples_split=2,min_samples_leaf=1,max_depth=1,random_state=42 )
gbc_fit.fit(X_train,y_train)

#print ("\nGradient Boost - Train Confusion Matrix\n\n",pd.crosstab(y_train,gbc_fit.predict(X_train),rownames = ["Actuall"],colnames = ["Predicted"]))      
print ("\nGradient Boost - Train accuracy",round(accuracy_score(y_train,gbc_fit.predict(X_train)),3))
#print ("\nGradient Boost  - Train Classification Report\n",classification_report(y_train,gbc_fit.predict(X_train)))

#print ("\n\nGradient Boost - Test Confusion Matrix\n\n",pd.crosstab(y_test,gbc_fit.predict(X_test),rownames = ["Actuall"],colnames = ["Predicted"]))      
print ("\nGradient Boost - Test accuracy",round(accuracy_score(y_test,gbc_fit.predict(X_test)),3))
#print ("\nGradient Boost - Test Classification Report\n",classification_report(y_test,gbc_fit.predict(X_test)))



Gradient Boost - Train accuracy 0.909

Gradient Boost - Test accuracy 0.889


In [19]:
# Xgboost Classifier
import xgboost as xgb

xgb_fit = xgb.XGBClassifier(max_depth=2, n_estimators=5000, learning_rate=0.05)
xgb_fit.fit(X_train, y_train)

#print ("\nXGBoost - Train Confusion Matrix\n\n",pd.crosstab(y_train,xgb_fit.predict(X_train),rownames = ["Actuall"],colnames = ["Predicted"]))      
print ("\nXGBoost - Train accuracy",round(accuracy_score(y_train,xgb_fit.predict(X_train)),3))
#print ("\nXGBoost  - Train Classification Report\n",classification_report(y_train,xgb_fit.predict(X_train)))

#print ("\n\nXGBoost - Test Confusion Matrix\n\n",pd.crosstab(y_test,xgb_fit.predict(X_test),rownames = ["Actuall"],colnames = ["Predicted"]))      
print ("\nXGBoost - Test accuracy",round(accuracy_score(y_test,xgb_fit.predict(X_test)),3))
#print ("\nXGBoost - Test Classification Report\n",classification_report(y_test,xgb_fit.predict(X_test)))


XGBoost - Train accuracy 0.998

XGBoost - Test accuracy 0.951


In [20]:
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

# Choose the type of classifier. 
clf = DecisionTreeClassifier()


# Choose some parameter combinations to try
parameters = {'max_features': ['log2', 'sqrt','auto'], 
              'criterion': ['entropy', 'gini'],
              'max_depth': [i for i in range(1,20)], 
              'min_samples_split': [i for i in range(2,10)],
              'min_samples_leaf': [i for i in range(2,10)]
             }

# Type of scoring used to compare parameter combinations
acc_scorer = make_scorer(accuracy_score)

# Run the grid search
grid_obj = GridSearchCV(clf, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(X_train, y_train)

# Set the clf to the best combination of parameters
clf = grid_obj.best_estimator_

# Fit the best algorithm to the data. 
print(clf.fit(X_train, y_train))



C:\Users\Venkat\Anaconda3_1\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=13,
                       max_features='log2', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=9, min_samples_split=7,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')


In [23]:
#Predict target value and find accuracy score
y_pred_train = clf.predict(X_train)
print("Accuracy score of train is ",accuracy_score(y_train, y_pred_train))
y_pred = clf.predict(X_test)
print("Accuracy score of test is ",accuracy_score(y_test, y_pred))

Accuracy score of train is  0.9508627156789198
Accuracy score of test is  0.9430284857571214
